In [120]:
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis,mode
from joblib import Parallel,delayed
import zipfile
import shutil
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
seed = 100
tf.random.set_seed(seed)
np.random.seed(seed)
import os
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split,LeavePGroupsOut
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,InputLayer,MaxPooling1D,Flatten,Dense,Input,Activation,GRU,Bidirectional,LSTM
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import accuracy_score
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings('ignore')

In [121]:
def get_X_y_groups(use_standardization,n_lag):
    feature_names = ['Time of Day','Gender', 'Age', 'current_window_stress_p90', 'current_window_stress_p95', 'current_window_stress_p80',
     'current_window_stress_p5', 'current_window_stress_p10', 'current_window_stress_p20', 'current_window_stress_range_90_to_80',
     'current_window_stress_range_20_to_10', 'current_window_stress_range_90_to_10', 'current_window_stress_range_80_to_20', 'current_window_stress_median',
     'current_window_stress_iqr', 'current_window_stress_skew', 'current_window_stress_diff_p90', 'current_window_stress_diff_p95',
     'current_window_stress_diff_p80', 'current_window_stress_diff_p5', 'current_window_stress_diff_p10', 'current_window_stress_diff_p20',
     'current_window_stress_diff_range_90_to_80', 'current_window_stress_diff_range_20_to_10', 'current_window_stress_diff_range_90_to_10', 'current_window_stress_diff_range_80_to_20',
     'current_window_stress_diff_median', 'current_window_stress_diff_iqr', 'current_window_stress_diff_skew', 'time_since_last_visit_smoking_spot',
     'duration_of_stay_in_smoking_spot', 'fraction_of_minutes_spent_in_smoking_spots_out_of_observed', 'daily_stress_p90', 'daily_stress_p95',
     'daily_stress_p80', 'daily_stress_p5', 'daily_stress_p10', 'daily_stress_p20', 'daily_stress_range_90_to_80', 'daily_stress_range_20_to_10',
     'daily_stress_range_90_to_10', 'daily_stress_range_80_to_20', 'daily_stress_median', 'daily_stress_iqr', 'daily_stress_skew', 'percentage_of_stress', 'maximum_duration_of_current_stress_episode',
     'average_duration_of_current_stress_episode', 'maximum_density_of_current_stress_episode', 'average_density_of_current_stress_episode', 'average_deviation_to_daily_mean_current',
     'no_stress_till_now', 'time_since_last_stress', 'duration_of_last_stress_episode', 'average_duration_of_before_stress_episode', 'density_of_last_stress_episode',
     'average_density_of_before_stress_episode', 'deviation_to_daily_mean_of_last_stress_episode', 'percentage_of_stress_before', 'percentage_of_active_',
     'maximum_duration_of_current_activity_episode', 'average_duration_of_current_window_episode', 'no_activity_till_now', 'time_since_last_activity',
     'duration_of_last_activity_episode', 'average_duration_of_before_activity_episode', 'percentage_of_active_before', 'is_smoking', 'spread', 'distance_to_nearest_spot', 'time_spent_in_transition',
     'time_spent_in_smoking_spot']
    if use_standardization:
        data  = pickle.load(open('./data/lagged_data/obs_30_prediction_60/lagged_'+str(n_lag)+'_windows.p','rb'))
    else:
        data  = pickle.load(open('./data/lagged_data/obs_30_prediction_60/lagged_'+str(n_lag)+'_windows'+'_standardized_new'+'.p','rb'))
    n_t = data.iloc[0]['features'].shape[1]
    n_f = data.iloc[0]['features'].shape[2]
    y_time = data['time'].values
    X = np.concatenate(list(data['features']))
    n = X.shape[0]
    X1 = X
    y = data['label'].values
    y = np.int64(np.array(y))
    y[y>0] = 1
    y[y<1] = -1
    groups = data['user'].values
    X_time = data['Time of Day'].values.reshape(-1,1)
    X_gender = data.Gender.values.reshape(-1,1)
    time_oh = preprocessing.OneHotEncoder().fit(X_time)
    gender_oh = preprocessing.OneHotEncoder().fit(X_gender)
    X_time = time_oh.transform(X_time).todense()
    X_gender = gender_oh.transform(X_gender).todense()
    if use_standardization:
        X1_all,y_all,groups_all,y_time_all,X_gender_all,X_time_all = [],[],[],[],[],[]
        for g in np.unique(groups):
            X1_all.append(X1[groups==g])
            y_all.extend(list(y[groups==g]))
            groups_all.extend(list(groups[groups==g]))
            y_time_all.extend(list(y_time[groups==g]))
            X_gender_all.append(preprocessing.StandardScaler().fit_transform(X_gender[groups==g]))
            X_time_all.append(preprocessing.StandardScaler().fit_transform(X_time[groups==g]))
        X1,y,groups,y_time,X_gender,X_time =  np.concatenate(X1_all),np.array(y_all),np.array(groups_all), \
        np.array(y_time_all),np.concatenate(X_gender_all),np.concatenate(X_time_all)

    X_time = np.concatenate([np.expand_dims(X_time,axis=1)]*n_t,axis=1)
    X_gender = np.concatenate([np.expand_dims(X_gender,axis=1)]*n_t,axis=1)
    feature_names_temp = feature_names[2:]
    time_feature_columns = [feature_names[0]+'=='+a for a in time_oh.categories_[0]]
    gender_feature_columns = [feature_names[1]+'=='+a for a in gender_oh.categories_[0]]
    feature_names_final = time_feature_columns+gender_feature_columns+feature_names_temp
    X = np.concatenate([X1,X_time,X_gender],axis=2)
    return X,y,groups,feature_names_final,y_time

def get_train_test_indexes(groups,n_groups_split = 10):
    groups_unique = np.unique(groups)
    groups_split = np.array_split(groups_unique,n_groups_split)
    indexes = []
    for this_groups in groups_split:
        this_groups = set(this_groups)
        train_index,test_index = np.array([i for i,a in enumerate(groups) 
                                           if a not in this_groups]),np.array([i for i,a in enumerate(groups) 
                                                                               if a in this_groups])
        indexes.append([train_index,test_index])
    return indexes



def get_model(X_train):
    n_t,n_f = X_train.shape[1],X_train.shape[2]
#     a = tf.placeholder(dtype=tf.float32, shape=(None, n_t,n_f))
    model = Sequential()
#     model.add(Input(shape=(n_t,n_f)))
#     model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(Conv1D(64,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(LSTM(80,activation='tanh',return_sequences=True,input_shape=(n_t,n_f)))
#     model.add(LSTM(20,activation='relu',return_sequences=True))
    model.add(LSTM(20,activation='tanh',return_sequences=False,input_shape=(n_t,n_f)))
    model.add(Dropout(.3))
    model.add(Flatten())
    model.add(Dense(10,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss=tfa.losses.focal_loss.SigmoidFocalCrossEntropy(),metrics=['acc'])
    return model

In [ ]:
use_standardization = True
n_lag = 10
n_groups = 20
X,y,groups,feature_names,y_time = get_X_y_groups(use_standardization,n_lag)
y[y<0] = 0
y = np.float32(y)
indexes = get_train_test_indexes(groups,n_groups_split = 10)
final_y_time = []
final_probs = []
final_y = []
final_groups = []
for train_index,test_index in indexes:
    X_train,X_test,y_train,y_test,groups_train,groups_test,time_train,time_test = X[train_index],X[test_index], \
    y[train_index],y[test_index],groups[train_index],groups[test_index],y_time[train_index],y_time[test_index]
    positive_train,negative_train,pos_train_y,neg_train_y = X_train[y_train==1],X_train[y_train==0],y_train[y_train==1],y_train[y_train==0]
    len_positive = len(positive_train)
    index = np.arange(len(negative_train))
    n_iters = 10
    test_preds = []
    for i,n_iter in enumerate(range(n_iters)):
        indexes_sampled = np.random.choice(index,len_positive*3)
        negative_train_sampled = negative_train[indexes_sampled]
        train_x = np.concatenate([positive_train,negative_train_sampled])
        train_y = np.array([1]*len(positive_train) + [0]*len(negative_train_sampled))
        model = get_model(train_x)
        filepath = './models/temp_model.hdf5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min',save_weights_only=False)
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,patience=40)
        callbacks_list = [es,checkpoint]
        train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=.1,stratify=train_y)
        train_y = tf.cast(train_y, tf.float32)
        train_x = tf.cast(train_x, tf.float32)
        val_y = tf.cast(val_y, tf.float32)
        val_x = tf.cast(val_x, tf.float32)
        history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=200, batch_size=30,
                            verbose=0,callbacks=callbacks_list,shuffle=True)
        model.load_weights(filepath)
        test_preds.append(model.predict(X_test))
    y_test_pred = list(np.concatenate(test_preds,axis=1).mean(axis=1))
    final_y_time.extend(list(time_test))
    final_probs.extend(list(y_test_pred))
    final_y.extend(list(y_test))
    final_groups.extend(list(groups_test))
    print(len(np.unique(final_groups)))
final_y_time,final_probs,final_y,final_groups = np.array(final_y_time),np.array(final_probs),np.array(final_y),np.array(final_groups)
pickle.dump([final_y_time,final_probs,final_y,final_groups],open('./data/output/all_features_single_LSTM.p','wb'))

((8489,), (8489,), (8489,), (8489,))

In [112]:
len(np.unique(final_groups))

16

In [ ]:
model = get_model(X_train)

In [ ]:
model.summary()

In [ ]:
import uuid
filepath = './models/'+'-'.join([str(n_lag),str(n_groups)])+'-'+str(uuid.uuid4())+'.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,patience=40)
callbacks_list = [es,checkpoint]
train_x,val_x,train_y,val_y = train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=100, batch_size=100,verbose=1,callbacks=callbacks_list,shuffle=True)

In [ ]:
model.load_weights(filepath)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import auc,roc_auc_score,f1_score
roc_auc_score(y_test,y_pred)

In [ ]:
len(np.unique(groups))

In [ ]:
y_train

In [ ]:
len